_Autor:_    __Jesús Casado__ <br> _Revisión:_ __02/07/2019__ <br>


__Descripción__:<br>
Funciones para extraer, combinar y agregar las series de datos originales del SAIH Cantábrico.

__Cosas a corregir__ <br>



In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta

In [16]:
def SAIH_CHC2(estacion, rutaorig, freq=None, export_orig=False, rutaexp=None,
                  verbose=True):
    """Genera las series diarias para las estaciones del SAIH Cantábrico.
    
    Entradas:
    ---------
    estacion:    str o int. Nombre de la estación
    ruta:        str.
    freq:        str. Resolución temporal a la que remuestrear los datos. Por defecto es 'None', es decir, se genera una serie con la resolución original cincominutal
    export_orig: boolean. Si exportar o no la serie original cincominutal
    verbose:     boolean.
    
    Salidas:
    --------
    Genera un archivo .csv con la serie de la estación indicada con la resolución temporal indicada (o 5 min si 'freq' es None). Si 'freq' es distinto de None y 'export_orig' es True, se exporta la serie cincominutal además de la serie de frecuencia indicada.
    Los archivos se guardan en una subcarpeta dentro de 'ruta' de nombre igual a la frecuencia de la serie.
    """

    # rutas
    rutaSAIH1 = rutaorig + '/Hasta junio de 2015/'
    rutaSAIH2 = rutaorig + '/Desde julio de 2015/'

    signames = {'AIPCINC': 'precipitacion_mm',
                'ACQRIO1': 'caudal_m³/s',
                'AINRIO1': 'nivel_m',
                'AINRL7S': 'nivel_m',
                'AITEMEX': 'temperatura_ºC',
                #'_PIEZO': 'piezometro_m',
                #'_POZO': 'piezometro_m',
                #'_LIMNI': 'limnimetro_m',
                'AIA3ATS': 'amonio_mg/l',
                'AIMPCTS': 'conductividad_μS/cm',
                'AIMPO2S': 'oxigeno_mg/l',
                'AIMPPHS': 'pH',
                'AIMPTTS': 'temperaturaAgua_ºC',
                'AITUTUS': 'turbidez_NTU'}
    redondeo = {'precipitacion_mm': 1, 'caudal_m³/s': 2, 'nivel_m': 2, 'temperatura_ºC': 3,
                'amonio_mg/l': 2, 'conductividad_μS/cm': 0, 'oxigeno_mg/l': 1, 'pH': 1,
                'temperaturaAgua_ºC': 1, 'turbidez_NTU': 0}# 'piezometro_m': 3, 'limnimetro_m': 3, 
    errormin = {'precipitacion_mm': 0, 'caudal_m³/s': 0, 'nivel_m': 0, 'temperatura_ºC': -50,
                'amonio_mg/l': 0, 'conductividad_μS/cm': 0, 'oxigeno_mg/l': 0, 'pH': 0,
                'temperaturaAgua_ºC': -10, 'turbidez_NTU': 0}# 'piezometro_m': 0, 'limnimetro_m': 0, 
    errormax = {'precipitacion_mm': 500, 'caudal_m³/s': 2000, 'nivel_m': 20, 'temperatura_ºC': 50,
                'amonio_mg/l': 1e4, 'conductividad_μS/cm': 1e6, 'oxigeno_mg/l': 1e3, 'pH': 15,
                'temperaturaAgua_ºC': 50, 'turbidez_NTU': 1e2}# 'piezometro_m': 1e3, 'limnimetro_m': 1e3, 
    
    # PARTE 1
    # -------
    # encontrar archivos de la estación
    folders = os.listdir(rutaSAIH1)
    for f, folder in enumerate(folders):
        if folder[:4] == str(estacion):
            ruta_stn = rutaSAIH1 + folder + '/'
            files = [file for file in os.listdir(ruta_stn) if file[:4] == folder[:4]]

            # Importar datos cincominutales
            data1 = pd.DataFrame()
            for i, file in enumerate(files):
                # importar serie original
                aux = pd.read_csv(ruta_stn + file, sep=';', encoding='latin-1', decimal=',', low_memory=False, na_values=[-100, 65535, 6523.6, -816])
                aux.dropna(axis=0, how='all', inplace=True)
                aux.Fecha = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux.Fecha]
                aux.set_index('Fecha', drop=True, inplace=True)
                # reordenar 'aux' por señales
                signals = [signal for signal in aux['Nombre señal'].unique() if signal[-7:] in list(signames.keys())]
                aux2 = pd.DataFrame()#columns=cols)
                for signal in signals:
                    temp = aux.loc[aux['Nombre señal'] == signal, 'Valor']
                    cols = list(aux2.columns)
                    aux2 = pd.concat((aux2, temp), axis=1, sort=True)
                    aux2.columns = cols + [str(estacion) + 'X' + signal[-7:]]
                # concatenar a la serie generada
                data1 = pd.concat((data1, aux2), axis=0, sort=True)
            del files, folders
            break
    
    # PARTE 2
    # -------
    # encontrar carpeta de la estación
    folders = os.listdir(rutaSAIH2)
    for folder in folders:
        if folder[:4] == str(estacion):
            ruta_stn = rutaSAIH2 + folder + '/'
            # encontrar archivos de la estación
            files = os.listdir(ruta_stn)
            # corregir nombre del archivo si fuera necesario
            for i, file in enumerate(files):
                if len(file) > 16:
                    new_file = file[:12] + file[-4:]
                    try:
                        os.rename(ruta_stn + file, ruta_stn + new_file)
                        files[i] = new_file
                    except:
                        continue
            # Importar datos cincominutales
            data2 = pd.DataFrame()
            for file in files:
                aux = pd.read_csv(ruta_stn + file, sep=';', decimal=',', encoding='latin-1', skiprows=1)
                aux['Fecha/Hora'] = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux['Fecha/Hora']]
                aux.set_index('Fecha/Hora', drop=True, inplace=True)
                aux.index.name = 'Fecha'
                #cols = [col for col in aux.columns if col[-7:] in list(signames.keys())]
                cols = [col for col in aux.columns if col[5:] in list(signames.keys())]
                aux = aux.loc[:, cols]
                data2 = pd.concat((data2, aux), axis=0, sort=True)
            del files, folders   
            break
    
    # UNIR SERIES
    # -----------
    # unir las dos series como serie minutal para evitar errores
    if ('data1' in locals()) and ('data2' in locals()):
        idx = pd.date_range(data1.index[0], data2.index[-1], freq='min')
        cols = list(data1.columns) + list(set(data2.columns) - set(data1.columns))
        data = pd.DataFrame(index=idx, columns=cols)
        data.loc[data2.index,:] = data2
        data.loc[data1.index,:] = data1
    elif ('data1' in locals()) and ('data2' not in locals()):
        idx = pd.date_range(data1.index[0], data1.index[-1], freq='min')
        data = pd.DataFrame(index=idx, columns=data1.columns)
        data.loc[data1.index,:] = data1
    elif ('data2' in locals()) and ('data1' not in locals()): 
        idx = pd.date_range(data2.index[0], data2.index[-1], freq='min')
        data = pd.DataFrame(index=idx, columns=data2.columns)
        data.loc[data2.index,:] = data2.loc[:, data.columns]
    
    # Agregar datos a la frecuencia de medición: 5 min
    data_5min = data.resample('5min').sum()
    # corregir nombre de las columnas
    data_5min.index.name = 'Fecha'
    data_5min.columns = [signames[col[5:]] for col in data_5min.columns if col[5:] in list(signames.keys())]
    #data_5min.columns = [signames[col[-7:]] for col in data_5min.columns if col[-7:] in list(signames.keys())]
    # corregir valores
    for col in data_5min.columns:
        mask = (data_5min[col] < errormin[col]) | (data_5min[col] > errormax[col])
        data_5min.loc[mask, col] = np.nan
    #for col in data_5min.columns:
    #    data_5min[col] = data_5min[col].round(redondeo[col])
    data_5min = data_5min.astype(float).round(redondeo)
    
    # remuestrear datos a la frecuencia deseada
    aux = data_5min.copy()  # copia de la serie obervada corregida
    if freq != None:
        data_ag = aux.resample(freq).sum()
        counts = aux.resample(freq).count()
        for col in data_ag.columns:
            if col in ['caudal_m³/s', 'nivel_m', 'temperatura_ºC']:
                data_ag[col] /= counts[col]
        #for col in data_ag.columns:
        #    data_ag[col] = data_ag[col].round(redondeo[col])
        data_ag = data_ag.astype(float).round(redondeo)

    # EXPORTAR
    # --------
    # exportar la serie cincominutal 
    if (export_orig == True) or (freq == None):
        ruta_5min = rutaexp + '5min/'
        if not os.path.exists(ruta_5min):
            os.makedirs(ruta_5min)
        data_5min.to_csv(ruta_5min + str(estacion) + '.csv', sep=',', na_rep='')#,
                         #float_format='%.3f')
    # exportar la serie remuestreada
    if freq != None:
        ruta_ag = rutaexp + freq + '/'
        if not os.path.exists(ruta_ag):
            os.makedirs(ruta_ag)
        data_ag.to_csv(ruta_ag + str(estacion) + '.csv', sep=',', na_rep='')#,
                       #float_format='%.3f')

    if verbose == True:
        print('nº de días de las serie:\t', data_ag.shape[0])
        print('variables:\t', list(data_ag.columns))

In [3]:
def SAIH_CHC2(estacion, rutaorig, freq=None, export_orig=False, rutaexp=None,
                  verbose=True):
    """Genera las series diarias para las estaciones del SAIH Cantábrico.
    
    Entradas:
    ---------
    estacion:    str o int. Nombre de la estación
    ruta:        str.
    freq:        str. Resolución temporal a la que remuestrear los datos. Por defecto es 'None', es decir, se genera una serie con la resolución original cincominutal
    export_orig: boolean. Si exportar o no la serie original cincominutal
    verbose:     boolean.
    
    Salidas:
    --------
    Genera un archivo .csv con la serie de la estación indicada con la resolución temporal indicada (o 5 min si 'freq' es None). Si 'freq' es distinto de None y 'export_orig' es True, se exporta la serie cincominutal además de la serie de frecuencia indicada.
    Los archivos se guardan en una subcarpeta dentro de 'ruta' de nombre igual a la frecuencia de la serie.
    """

    # rutas
    rutaSAIH1 = rutaorig + '/Hasta junio de 2015/' + str(estacion) + '/'
    rutaSAIH2 = rutaorig + '/Desde julio de 2015/' + str(estacion) + '/'

    signames = {'AIPCINC': 'precipitacion_mm',
                'ACQRIO1': 'caudal_m³/s',
                'AINRIO1': 'nivel_m',
                'AINRL7S': 'nivel_m',
                'AITEMEX': 'temperatura_ºC',
                #'_PIEZO': 'piezometro_m',
                #'_POZO': 'piezometro_m',
                #'_LIMNI': 'limnimetro_m',
                'AIA3ATS': 'amonio_mg/l',
                'AIMPCTS': 'conductividad_μS/cm',
                'AIMPO2S': 'oxigeno_mg/l',
                'AIMPPHS': 'pH',
                'AIMPTTS': 'temperaturaAgua_ºC',
                'AITUTUS': 'turbidez_NTU'}
    redondeo = {'precipitacion_mm': 1, 'caudal_m³/s': 2, 'nivel_m': 2, 'temperatura_ºC': 3,
                'amonio_mg/l': 2, 'conductividad_μS/cm': 0, 'oxigeno_mg/l': 1, 'pH': 1,
                'temperaturaAgua_ºC': 1, 'turbidez_NTU': 0}# 'piezometro_m': 3, 'limnimetro_m': 3, 
    errormin = {'precipitacion_mm': 0, 'caudal_m³/s': 0, 'nivel_m': 0, 'temperatura_ºC': -10,
                'amonio_mg/l': 0, 'conductividad_μS/cm': 0, 'oxigeno_mg/l': 0, 'pH': 0,
                'temperaturaAgua_ºC': -10, 'turbidez_NTU': 0}# 'piezometro_m': 0, 'limnimetro_m': 0, 
    errormax = {'precipitacion_mm': 500, 'caudal_m³/s': 2000, 'nivel_m': 20, 'temperatura_ºC': 50,
                'amonio_mg/l': 1e4, 'conductividad_μS/cm': 1e6, 'oxigeno_mg/l': 1e3, 'pH': 15,
                'temperaturaAgua_ºC': 50, 'turbidez_NTU': 1e2}# 'piezometro_m': 1e3, 'limnimetro_m': 1e3, 
    
    # PARTE 1
    # -------
    # encontrar archivos de la estación
    files = [file for file in os.listdir(rutaSAIH1) if file[:4] == str(estacion)]

    # Importar datos cincominutales
    data1 = pd.DataFrame()
    for i, file in enumerate(files):
        # importar serie original
        aux = pd.read_csv(rutaSAIH1 + file, sep=';', encoding='latin-1', decimal=',', low_memory=False, na_values=[-100, 65535, 6523.6, -816])
        aux.dropna(axis=0, how='all', inplace=True)
        aux.Fecha = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux.Fecha]
        aux.set_index('Fecha', drop=True, inplace=True)
        aux.loc[aux.Calidad == -6, 'Valor'] = np.nan # eliminar datos con baja calidad
        # reordenar 'aux' por señales
        signals = [signal for signal in aux['Nombre señal'].unique() if signal[-7:] in list(signames.keys())]
        aux2 = pd.DataFrame()#columns=cols)
        for signal in signals:
            temp = aux.loc[aux['Nombre señal'] == signal, 'Valor']
            cols = list(aux2.columns)
            aux2 = pd.concat((aux2, temp), axis=1, sort=True)
            aux2.columns = cols + [str(estacion) + 'X' + signal[-7:]]
        # concatenar a la serie generada
        data1 = pd.concat((data1, aux2), axis=0, sort=True)
    del files
    # corregir valores erróneos
    for col in data1.columns:
        mask = (data1[col] < errormin[signames[col[5:]]]) | (data1[col] > errormax[signames[col[5:]]])
        data1.loc[mask, col] = np.nan
    
    # PARTE 2
    # -------
    # encontrar archivos de la estación
    files = os.listdir(rutaSAIH2)
    # corregir nombre del archivo si fuera necesario
    for i, file in enumerate(files):
        if len(file) > 16:
            new_file = file[:12] + file[-4:]
            try:
                os.rename(ruta_stn + file, ruta_stn + new_file)
                files[i] = new_file
            except:
                continue
    # Importar datos cincominutales
    data2 = pd.DataFrame()
    for file in files:
        aux = pd.read_csv(rutaSAIH2 + file, sep=';', decimal=',', encoding='latin-1', skiprows=1)
        aux['Fecha/Hora'] = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux['Fecha/Hora']]
        aux.set_index('Fecha/Hora', drop=True, inplace=True)
        aux.index.name = 'Fecha'
        #cols = [col for col in aux.columns if col[-7:] in list(signames.keys())]
        cols = [col for col in aux.columns if col[5:] in list(signames.keys())]
        aux = aux.loc[:, cols]
        data2 = pd.concat((data2, aux), axis=0, sort=True)
    del files
    # corregir valores erróneos
    for col in data2.columns:
        mask = (data2[col] < errormin[signames[col[5:]]]) | (data2[col] > errormax[signames[col[5:]]])
        data2.loc[mask, col] = np.nan
    
    # UNIR SERIES
    # -----------
    # unir las dos series como serie minutal para evitar errores
    if ('data1' in locals()) and ('data2' in locals()):
        data = pd.concat((data1, data2), axis=0, sort=True)
    elif ('data1' in locals()) and ('data2' not in locals()):
        data = data1
    elif ('data2' in locals()) and ('data1' not in locals()): 
        data = data2
    
    # combinar columnas con la misma variable
    # variables disponibles
    variables = [signames[col[5:]] for col in data.columns]
    # encontrar variables repetidas
    var = list(set(variables))
    varName = [v for v in var if np.sum([v == variable for variable in variables]) > 1]
    # posición y nombre de las columnas a corregir
    for name in varName:
        colPos = [i for i, v in enumerate(variables) if v == name]
        colName = data.columns[colPos]
        # definir la columna fuente y destino
        colSrc = data.loc[:, colName].notnull().sum().idxmin()
        colDst = data.loc[:, colName].notnull().sum().idxmax()
        # traspasar datos entre columnas y eliminar columna fuente
        mask = data[colSrc].notnull()
        data.loc[mask, colDst] = data.loc[mask, colSrc]
        data.drop(colSrc, axis=1, inplace=True)
        del colPos, colName, colSrc, colDst
        
    # corregir nombre de las columnas
    data.index.name = 'Fecha'
    data.columns = [signames[col[5:]] for col in data.columns if col[5:] in list(signames.keys())]
    
    # corregir valores
    for col in data.columns:
        mask = (data[col] < errormin[col]) | (data[col] > errormax[col])
        data.loc[mask, col] = np.nan
    if 'caudal_m³/s' in data.columns:
        mask = (data['nivel_m'] > 0) & (data['caudal_m³/s'] == 0)
        data.loc[mask, 'caudal_m³/s'] = np.nan
    
    # AGREGAR DATOS A LA FRECUENCIA DESEADA
    # -------------------------------------
    data_ag = data.resample(freq).mean()
    if 'precipitacion_mm' in data.columns:
        data_ag['precipitacion_mm'] *= data['precipitacion_mm'].resample(freq).count()
    data_ag = data_ag.astype(float).round(redondeo)
    
#     # remuestrear datos a la frecuencia deseada
#     aux = data_5min.copy()  # copia de la serie obervada corregida
#     if freq != None:
#         data_ag = aux.resample(freq).sum()
#         counts = aux.resample(freq).count()
#         for col in data_ag.columns:
#             if col in ['caudal_m³/s', 'nivel_m', 'temperatura_ºC']:
#                 data_ag[col] /= counts[col]
#         #for col in data_ag.columns:
#         #    data_ag[col] = data_ag[col].round(redondeo[col])
#         data_ag = data_ag.astype(float).round(redondeo)

    # EXPORTAR
    # --------
#     # exportar la serie cincominutal 
#     if (export_orig == True) or (freq == None):
#         ruta_5min = rutaexp + '5min/'
#         if not os.path.exists(ruta_5min):
#             os.makedirs(ruta_5min)
#         data_5min.to_csv(ruta_5min + str(estacion) + '.csv', sep=',', na_rep='')#,
#                          #float_format='%.3f')
    # exportar la serie remuestreada
    if (freq != None) and (rutaexp != None):
        ruta_ag = rutaexp + freq + '/'
        if not os.path.exists(ruta_ag):
            os.makedirs(ruta_ag)
        data_ag.to_csv(ruta_ag + str(estacion) + '.csv', sep=',', na_rep='', encoding='latin1')#,
                       #float_format='%.3f')

    if verbose == True:
        print('nº de días de las serie:\t', data_ag.shape[0])
        print('variables:\t', list(data_ag.columns))
        
    SAIH_CHC2.data = data_ag